In [5]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import json
from shapely import wkt, wkb
from shapely.geometry import Polygon
import contextily as ctx
import os
from pathlib import Path

def get_geom(row, t):
    """
    decode geometries and zero-out any invalid geometries
    """
    try:
        return wkb.loads(row[f'isochrone_{t}'], hex=True)
    except:
        return Polygon()
    
def get_opps(row, t):
    return float(row.get(f'opps_jobs_{t}', 0.0))

In [6]:
path = Path("/Users/rfitzger/data/mep/mep3/input/experiment/2025-05-19-cemal")
opps = pd.read_csv(path / "result.csv.gz")

In [7]:
if not os.path.isdir(path / "iso_img"):
    os.mkdir(path / "iso_img")

In [15]:
import numpy as np
times = [40,30,20,10]

i = 0
for _, row in opps.iterrows():
    hex_id = row['grid_id']
    mode = row['mode']
    plot_file = f'{path}/iso_img/isos-{hex_id}-{mode}.png'
    if os.path.exists(plot_file):
        pass
    row_geoms = [get_geom(row, t) for t in times]
    row_opps = [get_opps(row, t) for t in times]
    # try:
    # plot_df = gpd.GeoDataFrame({"geometry": row_geoms, "opps": row_opps}, crs="EPSG:4326")
    plot_df = gpd.GeoDataFrame({"geometry": row_geoms, "time": np.ndarray(times)}, crs="EPSG:4326")
    fig, ax = plt.subplots()
    # plot_df.to_crs("EPSG:3857").plot('opps', ax=ax, alpha=0.3, edgecolor='k', linewidth=0.25, cmap='cool')
    plot_df.to_crs("EPSG:3857").plot('time', ax=ax)
    ax.set_xticks([])
    ax.set_yticks([])
    ctx.add_basemap(ax=ax, source=ctx.providers.USGS.USTopo)
    fig.savefig(plot_file, dpi=300, bbox_inches='tight')
    # except e:
        
    #     pass
    plt.close()
    i += 1

ValueError: Per-column arrays must each be 1-dimensional

In [ ]:
i